In [134]:
from SE3Lie_ca import se3, SE3, dot_plot_draw, so3_wedge, so3_vee
import casadi as ca

#http://asrl.utias.utoronto.ca/~tdb/bib/barfoot_ser17.pdf

In [135]:
pts = ca.SX([1,2, 3, 0.1, 0.2, 0.3]) #true location [x y z theta1 theta2 theta3]
pt_cam = ca.SX([[1,-2,3],[1,2,3],[1,2,-3],[1,-2,-3],[-1,2,3],[-1,-2,-3],[-1,-2,3],[-1,2,-3]]) #Camera from corners in inertial frame

# pt_guess = ca.SX([])


In [136]:
def T_op_rt(T):
    '''
    Accepts T_op Matrix and split to R and t terms
    '''
    R_op = T[:3,:3]
    t_op = T[:3,3]
    # T_op = ca.vertcat(ca.horzcat(R_op, -R_op@t_op),ca.horzcat(ca.SX.zeros(1,R_op.shape[1]),1))
    return R_op,t_op

def Tau_op(T):
    '''
    Accepts SE3 elements operator (T_op) and find big Adjoint SE(3)
    '''
    return SE3.Ad(T)


# R_op = so3_wedge([0.1,0.2,0.3])#ca.SX.eye(3)
# t_op = ca.SX.zeros(3,1)
# t_op[0:3] = [1,2,3]
# T_op = ca.vertcat(ca.horzcat(R_op, -R_op@t_op),ca.horzcat(ca.SX.zeros(1,R_op.shape[1]),1))
# T_op

In [137]:
# wj = 1 #weight on each points

def p_sum(pt_cam):
    '''
    Accepts points from camera frame to origin and outputs 1x3 matrix for p location and w_sum
    '''
    p = 0
    w = 0
    for i in range(pt_cam.shape[0]):
        p += pt_cam[i,:]
        w += 1 #assume weight on each camera points is 1
    return ca.horzcat(p,[1]).T,w
p_sum(pt_cam)[0]

SX(@1=0, [@1, @1, @1, 1])

In [138]:
pt_cam[1,:]
pj = ca.horzcat(pt_cam[1,:],1).T
so3_wedge(pj-p_sum(pt_cam)[0])


SX(
[[00, -3, 2], 
 [3, 00, -1], 
 [-2, 1, 00]])

In [139]:
def I_term(pt_cam):
    '''
    Accepts matrix array of points from camera frame to origin and ouputs I_term
    '''
    I = ca.SX.zeros(3,1)
    P = p_sum(pt_cam)[0]
    # display(P)
    w = p_sum(pt_cam)[1]
    w = 0
    wj =1
    sum_term = ca.SX.zeros(3,3) 
    for i in range(pt_cam.shape[0]):
        pj = ca.horzcat(pt_cam[i,:],1).T
        sum_term += wj* so3_wedge(pj-P)@so3_wedge(pj-P)
        w += 1
        # display(sum_term)
    return - sum_term/w
## should pj term be 4x4?
I = I_term(pt_cam)
I

SX(@1=0, 
[[13, @1, @1], 
 [@1, 10, @1], 
 [@1, @1, 5]])

In [140]:
def M_mat(pt_cam):
    p = p_sum(pt_cam)[0]
    I = I_term(pt_cam)
    c1 = ca.vertcat( ca.horzcat(ca.SX.eye(3),ca.SX.zeros(3,3)) , ca.horzcat(so3_wedge(p),ca.SX.eye(3)))
    c2 = ca.vertcat( ca.horzcat(ca.SX.eye(3),ca.SX.zeros(3,3)) , ca.horzcat(ca.SX.zeros(3,3),I)  )
    c3 = ca.vertcat( ca.horzcat(ca.SX.eye(3),-so3_wedge(p)), ca.horzcat(ca.SX.zeros(3,3),ca.SX.eye(3)))
    return c1@c2@c3

M = M_mat(pt_cam)
M

SX(@1=1, @2=0, 
[[@1, @2, @2, @2, @2, @2], 
 [@2, @1, @2, @2, @2, @2], 
 [@2, @2, @1, @2, @2, @2], 
 [@2, @2, @2, 13, @2, @2], 
 [@2, @2, @2, @2, 10, @2], 
 [@2, @2, @2, @2, @2, 5]])

In [141]:
def calc_y(pt_cam):
    '''
    Accepts camera points and points of body frame and return the camera frame location from body to each camera
    '''
    true_loc = [1, 2, 3, 0.1, 0.2, 0.3] #True position in [x y z theta1 theta2 theta3]
    R_t = true_loc[3:] #True rotational terms
    t_t = true_loc[:3] #true translational terms
    T_true = se3.exp(se3.wedge(true_loc))
    # T_true[1,2] = T_true[1,2]/2
    w= 0
    yj= ca.SX.zeros(8,4)
    # display(T_true)
    for i in range(pt_cam.shape[0]):
        pj = ca.horzcat(pt_cam[i,:],1).T
        yj[i,0:4] = T_true@pj
        w += 1 #assume weight on each camera points is 1
    return yj, w

calc_y(pt_cam)[0]

SX(@1=1, 
[[3.13266, 0.197678, 5.49066, @1], 
 [2, 4, 6, @1], 
 [0.73885, 4.40819, 0.148258, @1], 
 [1.87151, 0.605865, -0.36108, @1], 
 [0.12849, 3.39413, 6.36108, @1], 
 [1.11022e-16, -2.22045e-16, 0, @1], 
 [1.26115, -0.408188, 5.85174, @1], 
 [-1.13266, 3.80232, 0.509338, @1]])

In [142]:
# ca.horzcat(test.T,(ca.SX.zeros(test.shape[0],1))).T

def y_sum(pt_cam): #use calc_y function for pt_y
    '''
    Accepts measured points from body frame and outputs 1x3 matrix for p location and w_sum
    y_sum term eqn 8.105
    '''
    y = 0
    w = 0
    dist_y = calc_y(pt_cam)[0]
    for i in range(dist_y.shape[0]):
        y += dist_y[i,:]
        w += 1      #assume weight on each camera points is 1
    return y.T/w,w

y_s = y_sum(pt_cam)
y_s
# pt_guess
# calc_y(pt_cam,pt_guess)[1,:]

(SX([1, 2, 3, 1]), 8)

In [143]:
def W_term(pt_cam):
    '''
    Accepts matrix array of points from camera to origin inertial frame and y distance in body frame **FIX**
    Accepts camera points and poitns of body frame in inertial frame and calculate W_term
    and ouputs W_term
    '''
    I = ca.SX.zeros(3,1)
    P = p_sum(pt_cam)[0]
    w = p_sum(pt_cam)[1]
    yj = calc_y(pt_cam)[0].T
    Y = y_sum(pt_cam)[0] #sum of y

    W = 0
    sum_term = ca.SX.zeros(4,4)
    for i in range(pt_cam.shape[0]):
        pj = ca.horzcat(pt_cam[i,:],1).T
        sum_term += (yj[:,i]-Y)@ca.transpose(pj-P)
        W += 1
    return sum_term/W
W_term(pt_cam)

SX(@1=0, 
[[0.935755, -1.13266, 1.89173, @1], 
 [0.302933, 3.80232, -0.612282, @1], 
 [-0.18054, 0.509338, 8.77761, @1], 
 [@1, @1, @1, @1]])

In [144]:
# running loops until convergence


R_op = ca.SX.eye(3)
t_op = ca.SX.zeros(3,1)

# Initial = ca.SX.zeros(6,1)
# R_op= so3_wedge(Initial[3:])
# t_op = Initial[:3]

T_op1 = ca.vertcat(ca.horzcat(R_op, -R_op.T@t_op),ca.horzcat(ca.SX.zeros(1,R_op.shape[1]),1))
T_op1


SX(@1=1, @2=0, 
[[@1, 00, 00, @2], 
 [00, @1, 00, @2], 
 [00, 00, @1, @2], 
 [@2, @2, @2, @1]])

In [145]:
def b_term(pt_cam, Top):
    W_t = ca.transpose( W_term(pt_cam)[:3,:3] ) #3x3 of W_terms
    C_op_term = Top[:3,:3]
    ident = ca.SX.eye(W_t.shape[0]) #Check what one should be
    b = ca.SX.zeros(1,3)
    for i in range(3):
        b[i] = ca.trace(so3_wedge(ident[i,:])@C_op_term@W_t)
    return b

b_term(pt_cam, T_op1)

def find_a(pt_cam,Top):
    # y = calc_y(pt_cam)[0]
    y = y_sum(pt_cam)[0]
    C_op_term = Top[:3,:3]
    r_op_term  = -C_op_term@Top[:3,3] #####
    P = p_sum(pt_cam)[0]
    b = b_term(pt_cam,Top)
    
    t1 = y[:3] - C_op_term@(P[:3]-r_op_term) #rop and Cop already calculated for Top
    t2 = b.T - so3_wedge(y)@C_op_term@(P[:3]-r_op_term)
    return ca.vertcat(t1,t2)
    # t1 = y[:3] - C_op_term@(P[:3]-r_op_term) #check if y supposed to be [:3]
    # t2 = b.T - so3_wedge(y)@C_op_term@(P[:3]-r_op_term)
    # return ca.vertcat(t1,t2)


# l1 = find_a(pt_cam,T_op1)
# l1

In [146]:
counter = 0
T_last1 = T_op1
while counter < 5:
    a_sum = find_a(pt_cam,T_op1)
    TauOp1 = SE3.Ad(T_op1)
    eps_st = TauOp1 @ ca.inv(M_mat(pt_cam)) @ TauOp1.T @ a_sum
    T_op1 = se3.exp(se3.ad_matrix(eps_st)) @ T_last1
    T_last1 = T_op1
    counter += 1

so3_vee(T_op1[:3,:3])


SX([-0.344787, -0.0465596, 0.623339])

In [147]:
T_op1[:3,3]

SX([7679.25, -12918.4, 11264])

In [148]:
#Initialize T_op
# R_op = so3_wedge([0.1,0.2,0.3]) #so3_wedge(ca.SX.zeros(3,1))#ca.SX.eye(3)
R_op = ca.SX.eye(3)
t_op = ca.SX.zeros(3,1)

T_op = ca.vertcat(ca.horzcat(R_op, -R_op@t_op),ca.horzcat(ca.SX.zeros(1,R_op.shape[1]),1))
T_op

SX(@1=1, @2=0, 
[[@1, 00, 00, @2], 
 [00, @1, 00, @2], 
 [00, 00, @1, @2], 
 [@2, @2, @2, @1]])

In [149]:
### another way to calculate a using summation method

def a_new_sum(pt_cam, Top):
    '''
    Accepts points to camera in inertial frame and p guess calcualte a matrix
    '''

    I = ca.SX.zeros(3,1)
    p = p_sum(pt_cam)[0]
    w = 0
    yj = calc_y(pt_cam)[0].T
    y = y_sum(pt_cam)[0] #sum of y
    pj = ca.horzcat(pt_cam,ca.SX.ones(pt_cam.shape[0],1)).T
    sum_terms = ca.SX.zeros(6,1)
    for i in range(pt_cam.shape[0]):
        zj = Top@(pj[:,i])
        # print(ca.horzcat(zj[3]@ca.SX.eye(3),-1*so3_wedge(zj[:3])))
        zj_circle = (ca.vertcat( 
            ca.horzcat(zj[3]@ca.SX.eye(3),-1*so3_wedge(zj[:3])),
            ca.SX.zeros(1,6))) #circle notation from barfoot 4x6 matrix
        sum_terms += zj_circle.T @ ((yj[:,i])-zj)
        w += 1
    return sum_terms / w

a_new_sum(pt_cam, T_op)

SX([1, 2, 3, 1.12162, 2.07227, 1.43559])

In [150]:
## Test iteration using summation of a
counter = 0
T_last = T_op
while counter < 10:
    a_sum = a_new_sum(pt_cam,T_op)
    Tau = Tau_op(T_op)
    eps_st = Tau @ ca.inv(M_mat(pt_cam)) @ Tau.T @ a_sum
    T_op = se3.exp(se3.ad_matrix(eps_st)) @ T_last
    T_last = T_op
    counter += 1
# T_op

In [151]:
SE3.log(T_op)

SX(@1=0, 
[[00, -0.3, 0.2, 19.2202], 
 [0.3, 00, -0.1, 91.2597], 
 [-0.2, 0.1, 00, -60.3246], 
 [@1, @1, @1, @1]])

In [152]:
angles = so3_vee(T_op[:3,:3])
angles

SX([0.127335, 0.210192, 0.302933])